In [1]:
import os
import re
import json
import fitz  # PyMuPDF
import unicodedata
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Tuple
import time

INPUT_DIR = Path('data/pdf/categorized/simplified')
OUTPUT_DIR = Path('data/json/simplified')

In [2]:
# TODO: remover "s" do final de todas as palavras


In [3]:

print(f"📂 Diretório de entrada: {INPUT_DIR}")
print(f"📂 Diretório de saída: {OUTPUT_DIR}")

# Verificar se existem arquivos
arquivos_pdf = list(INPUT_DIR.glob("*.pdf"))
print(f"📄 Total de arquivos PDF encontrados: {len(arquivos_pdf)}")

if arquivos_pdf:
    print("\n📋 Primeiros 5 arquivos:")
    for i, arquivo in enumerate(arquivos_pdf[:5]):
        print(f"   {i+1}. {arquivo.name}")

📂 Diretório de entrada: data\pdf\categorized\simplified
📂 Diretório de saída: data\json\simplified
📄 Total de arquivos PDF encontrados: 1567

📋 Primeiros 5 arquivos:
   1. A-076CENIPA2018_PUJYZ_atualizadoRFS_MTA.pdf
   2. A-135CENIPA2019_PRZJO_15ABR2021.pdf
   3. BRXXX2_28_07_2024_PUB..pdf
   4. CJB_encdivulgacao.pdf
   5. COBT_encdivulgacao.pdf


In [4]:
# Funcao para limpar hard os textos

def limpeza_hard(texto: str) -> str:
    """
    Padroniza texto:
    - deixa tudo minusculo,
    - remove acentuacao, pontos, espacos multiplos, linhas em branco,
    palavras com dois caracteres ou menos e olho gordo
    """

    texto = texto.lower().replace(".", " ")

    # troca multiplos espacos/tabs/quebras por 1 espaco
    texto = re.sub(r"[ \t]+", " ", texto)
    # remove acentuacao das palavras
    texto = "".join(
        c
        for c in unicodedata.normalize("NFD", texto)
        if unicodedata.category(c) != "Mn"
    )
    # remove tudo que nao eh numero ou caracter do texto
    texto = re.sub(r'[^a-z0-9\s]', ' ', texto)
    # remove palavras com 2 caracteres ou menos mantendo as quebras de linha e os numeros
    linhas = []
    for linha in texto.split("\n"):
        palavras = [
            p for p in linha.split()
            if len(p) > 2 or p.isdigit() or re.match(r'^\d+(\.\d+)*$', p)
        ]
        if palavras:
            linhas.append(" ".join(palavras))
    texto = "\n".join(linhas)

    return texto


# # =========== Teste
# path = "../dados/pdf/por_categoria/Simplificado/PT-EAE_13_04_2016-ACID.pdf"
# texto_raw = extrair_texto_pymupdf(path)
# texto_limpo = limpeza_hard(texto_raw)
# display(texto_raw)
# display(texto_limpo)

In [5]:
# Funções de extração de texto (COM FUNÇÃO PARA IGNORAR CABEÇALHO)

def extrair_texto_pymupdf(pdf_path: str) -> str:
    """
    Extrai texto de PDF usando PyMuPDF, preservando quebras de página.  
    Retorna o texto todo do arquivo como uma grande stringona
    """
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text() + "\n"
        doc.close()
        return text
    except Exception as e:
        print(f"❌ Erro ao extrair texto de {Path(pdf_path).name}: {e}")
        return ""

def ignorar_cabecalho_advertencia(texto_completo: str) -> str:
    """
    Encontra o início do conteúdo principal do relatório (após o bloco de advertência)
    e retorna apenas o texto relevante.
    Essa funcao roda antes do texto passar pela limpeza_hard
    """
    # Marcadores que indicam o início do conteúdo real do relatório
    marcadores_conteudo = [
        r'\d+\s+informacoes factuais',
        r'\d+\s+historico do voo',
        r'\d+\s+\d+\s+historico do voo'
    ]

    posicao_inicio = -1

    # Procura pelo primeiro marcador que aparece no texto
    for marcador in marcadores_conteudo:
        match = re.search(marcador, texto_completo, re.IGNORECASE)
        if match:
            if posicao_inicio == -1 or match.start() < posicao_inicio:
                posicao_inicio = match.start()

    if posicao_inicio != -1:
        # Retorna o texto a partir do marcador encontrado
        return texto_completo[posicao_inicio:]
    else:
        # Se nenhum marcador for encontrado, retorna o texto original e avisa o usuário.
        print(f"  ⚠️  Aviso: Início do conteúdo principal não identificado. A extração pode conter ruído do cabeçalho.")
        return texto_completo

def limpar_texto_secao(texto: str, fatores = False) -> str:
    """
    Limpeza específica para o conteúdo de uma seção.
    Remove quando a linha inteira é "01 de 04", ou "nnn cenipa yyyy"
    """
    if not texto:
        return ""
    linhas = [linha.strip() for linha in texto.strip().split('\n')]
    linhas_filtradas = []
    for linha in linhas:
        if (linha 
            and not re.match(r'^\d+\s+\d+$', linha)  # "nn nn"
            and not re.match(r'^\b\d{3}\s+cenipa\s+\d{4}\b$', linha)  # "xxx cenipa yyyy"
            and not re.match(r'^\d{1,2}[a-z]{3}\d{4}$', linha)  # ddmmyyyy
            ):
            linhas_filtradas.append(linha)
    if fatores:
        return '\n'.join(linhas_filtradas).strip()
    return ' '.join(linhas_filtradas).strip()

In [6]:
# Célula 6: Função para encontrar seções
def encontrar_secao(texto_completo: str, marcadores_inicio: List[str], marcadores_fim: List[str], fatores = False) -> str:
    """
    Encontra uma seção no texto baseada em marcadores de início e fim.
    Esta versão é mais robusta para as variações dos relatórios simplificados.
    """
    texto_upper = texto_completo
    inicio_secao = -1
    fim_marcador_inicio = -1

    # Encontra a posição do marcador de início
    for marcador in marcadores_inicio:
        match = re.search(marcador, texto_upper)
        if match:
            inicio_secao = match.start()
            fim_marcador_inicio = match.end()
            break

    if inicio_secao == -1:
        return ""

    # Encontra a posição do primeiro marcador de fim APÓS o início da seção
    fim_secao = len(texto_completo)
    for marcador in marcadores_fim:
        # Procuramos o marcador no resto do texto
        match = re.search(marcador, texto_upper[fim_marcador_inicio:])
        if match:
            # A posição encontrada é relativa ao texto cortado, então somamos `fim_marcador_inicio`
            posicao_absoluta = match.start() + fim_marcador_inicio
            if posicao_absoluta < fim_secao:
                fim_secao = posicao_absoluta
        # TODO se ele nao encontra ele vai ate o fim do documento, colocar um limite (ex 200 palavras)

    # Extrai o texto da seção e limpa
    secao_bruta = texto_completo[fim_marcador_inicio:fim_secao]
    # if fatores:
    #     return secao_bruta
    return limpar_texto_secao(secao_bruta, fatores)


In [7]:
# Célula 7: Funções específicas para cada seção

# 1. LISTA MESTRA COM TODOS OS TÍTULOS DE SEÇÃO POSSÍVEIS.
MARCADORES_DE_FIM_UNIVERSAIS = [
    r'\d+\s+historico voo',
    r'\d+\s+analise',
    r'\d+\s+comentarios',
    r'\d+\s+fatos',
    r'\d+\s+conclusoes',
    r'\d+\s+fatores contribuintes',
    r'\d+\s+recomendacoes seguranca',
    r'\d+\s+acoes corretivas',
    r'\d+\s+interrupcao investigacao',
    r'^\s*\d+(?:\s+\d+)*\s+[a-z]{4,}'
]

def extrair_historico_voo(texto: str) -> str:
    """Extrai a seção Histórico do voo"""
    # Marcadores de início podem ser mais flexíveis
    marcadores_inicio = [
        r'\d+\s+historico voo'
    ]
    return encontrar_secao(texto, marcadores_inicio, MARCADORES_DE_FIM_UNIVERSAIS)

def extrair_analise(texto: str) -> str:
    """Extrai a seção ANÁLISE ou equivalente (Comentários)"""
    marcadores_inicio = [
        r'\d+\s+analise',
        r'\d+\s+comentarios'
    ]
    return encontrar_secao(texto, marcadores_inicio, MARCADORES_DE_FIM_UNIVERSAIS)

def extrair_fatos(texto: str) -> str:
    """Extrai a seção Fatos"""
    marcadores_inicio = [
        r'\d+\s+fatos'
    ]
    return encontrar_secao(texto, marcadores_inicio, MARCADORES_DE_FIM_UNIVERSAIS)

def extrair_fatores_contribuintes(texto: str) -> str:
    """Extrai a seção Fatores contribuintes"""
    marcadores_inicio = [
        r'\d+\s+fatores contribuintes'
    ]
    return encontrar_secao(texto, marcadores_inicio, MARCADORES_DE_FIM_UNIVERSAIS, fatores = True)

In [8]:
# Célula 8: Processamento de fatores contribuintes (AJUSTADO PARA LIDAR COM A CONJUNÇÃO 'e')

def processar_fatores_contribuintes(texto_fatores: str) -> list[str]:
    """
    Processa o texto de fatores contribuintes para extrair o fator e sua classificação.
    Agora lida com a conjunção 'e' e múltiplos padrões de texto.
    """
    if not texto_fatores:
        return []

    linhas = texto_fatores.split('\n')
    
    fatores_lista = []
    for linha in linhas:
        linha_limpa = linha.strip()
        if not linha_limpa or len(linha_limpa) < 5:
            continue

        fator = ""
        # Padrão 2: Tenta encontrar "Fator Classificação" (sem hífen)
        match_sem_hifen = re.search(r'^(.*?)\s+(contribuiu|indeterminado|nao contribuiu)$', linha_limpa)
        if match_sem_hifen:
            # Divide a linha na última palavra para separar o fator da classificação
            partes = linha_limpa.rsplit(None, 1)
            fator = partes[0].strip()

        # Padrão 3: Se nenhum dos anteriores funcionar, a linha toda é o fator
        if not fator:
            fator = linha_limpa

        # Adiciona ao dicionário
        # if fator:
        if fator and not re.match(r'^\d+\s+\d+$', fator) and not re.match(r'^\b\d{3}\s+cenipa\s+\d{4}\b$', fator):
      
            fatores_lista.append(fator)

    return fatores_lista

In [9]:
# Célula 9: Função principal de processamento (AJUSTADA PARA USAR A LIMPEZA DE CABEÇALHO)


def processar_relatorio_simplificado(pdf_path: str) -> Dict[str, Any]:
    """
    Processa um relatório SIMPLIFICADO e extrai as informações necessárias
    """
    
    nome_arquivo = Path(pdf_path).name
    print(f"🔄 Processando: {nome_arquivo}")

    # 1. Extrai o texto bruto do PDF
    texto_bruto = extrair_texto_pymupdf(pdf_path)

    if not texto_bruto:
        print(f"  ⚠️  Não foi possível extrair texto do arquivo")
        return None

    # 2. IGNORA O CABEÇALHO, mantendo apenas o conteúdo principal

    texto_limpo = limpeza_hard(texto_bruto)

    texto_conteudo = ignorar_cabecalho_advertencia(texto_limpo)

    # 3. Extrai as seções a partir do texto já limpo
    historico = extrair_historico_voo(texto_conteudo)
    analise = extrair_analise(texto_conteudo)
    fatos = extrair_fatos(texto_conteudo)
    fatores_texto = extrair_fatores_contribuintes(texto_conteudo)
    fatores_lista = processar_fatores_contribuintes(fatores_texto)

    # Verificar se extraiu conteúdo relevante
    conteudos_extraidos = [historico, analise, fatos, fatores_lista]
    if not any(conteudos_extraidos):
        print(f"  ⚠️  Não foram encontradas seções relevantes em {nome_arquivo}")
        return None

    # Construir estrutura JSON
    resultado = {
        "metadata": {
            "arquivo_origem": nome_arquivo,
            "data_processamento": time.strftime("%Y-%m-%d %H:%M:%S"),
        },
        "textos": {
            "historico": historico,
            "analise": analise,
            "fatos": fatos
        },
        "labels": fatores_lista
    }

    print(f"  ✅ Extraído: H({len(historico)}), A({len(analise)}), F({len(fatos)}), L({len(fatores_lista)})")
    return resultado


In [10]:
# Célula 10: Processamento em lote (Corrigido)


def processar_todos_relatorios():
    """Processa TODOS os relatórios no diretório de entrada"""
    arquivos_pdf = list(INPUT_DIR.glob("*.pdf"))

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    if not arquivos_pdf:
        print(f"❌ Nenhum arquivo PDF encontrado em {INPUT_DIR}")
        return

    print(f"📁 Encontrados {len(arquivos_pdf)} arquivos PDF")
    print("🚀 Iniciando processamento...\n")

    resultados_sumario = []
    sucessos = 0
    falhas = 0

    for i, pdf_path in enumerate(arquivos_pdf):

        # if i > 100:  # ⚠️ ⚠️ ⚠️  COMENTAR ANTES DE RODAR PARA VALER
        #     break

        print(f"\n📄 [{i+1}/{len(arquivos_pdf)}] ", end="")
        try:
            resultado = processar_relatorio_simplificado(str(pdf_path))
            if resultado:
                # Salvar JSON individual
                nome_json = f"{str(i).zfill(4)}_" + pdf_path.stem + ".json"
                caminho_json = OUTPUT_DIR / nome_json

                with open(caminho_json, 'w', encoding='utf-8') as f:
                    json.dump(resultado, f, ensure_ascii=False, indent=4)

                # Adicionar ao sumário de resultados
                resultados_sumario.append({
                    'arquivo': pdf_path.name,
                    'json_salvo': f"{str(i).zfill(4)}_" + pdf_path.stem,
                    'historico_chars': len(resultado['textos']['historico']),
                    'analise_chars': len(resultado['textos']['analise']),
                    'fatos_chars': len(resultado['textos']['fatos']),
                    'fatores_count': len(resultado['labels'])
                })
                sucessos += 1
            else:
                print(f"Nao encontrei resultado ao processar_relatorio_simplificado {pdf_path.name}")
                falhas += 1

        except Exception as e:
            print(f"  ❌ Erro crítico ao processar {pdf_path.name}: {e}")
            falhas += 1

    # Salvar relatório de processamento
    if resultados_sumario:
        df_relatorio = pd.DataFrame(resultados_sumario)
        relatorio_path = OUTPUT_DIR.parent / "_relatorio_processamento.csv"
        df_relatorio.to_csv(relatorio_path, index=False, encoding='utf-8')

        print("\n" + "="*60)
        print("📊 RELATÓRIO DE PROCESSAMENTO FINAL")
        print("="*60)
        print(f"✅ Sucessos: {sucessos}")
        print(f"❌ Falhas: {falhas}")
        if arquivos_pdf:
          print(f"📈 Taxa de sucesso: {(sucessos/len(arquivos_pdf))*100:.1f}%")
        print(f"\n💾 JSONs salvos em: {OUTPUT_DIR}")
        print(f"📋 Relatório salvo em: {relatorio_path}")

    return resultados_sumario


In [11]:
# EXECUÇÃO PRINCIPAL


print("\n" + "="*60)
print("🚀 INICIANDO PROCESSAMENTO DOS RELATÓRIOS")
print("="*60)

# Processar todos os relatórios
resultados = processar_todos_relatorios()



🚀 INICIANDO PROCESSAMENTO DOS RELATÓRIOS
📁 Encontrados 1560 arquivos PDF
🚀 Iniciando processamento...


📄 [1/1560] 🔄 Processando: PT-EAE_13_04_2016-ACID.pdf
  ✅ Extraído: H(1437), A(0), F(865), L(0)

📄 [2/1560] 🔄 Processando: PR-KKR_19_06_2017-IG.pdf
  ✅ Extraído: H(2169), A(0), F(892), L(0)

📄 [3/1560] 🔄 Processando: PR-TOS_13_12_2019_AC..pdf
  ✅ Extraído: H(399), A(4189), F(802), L(3)

📄 [4/1560] 🔄 Processando: PT-NQS_25_10_2016_-_AC.pdf
  ✅ Extraído: H(4519), A(0), F(964), L(0)

📄 [5/1560] 🔄 Processando: pp_giv_01_12_2016.pdf
  ✅ Extraído: H(404), A(3331), F(770), L(6)

📄 [6/1560] 🔄 Processando: PT-GUT_06_02_2008-AC.pdf
  ✅ Extraído: H(498), A(1016), F(943), L(5)

📄 [7/1560] 🔄 Processando: PT-OIW_29_03_2014_-_ACID.pdf
  ✅ Extraído: H(582), A(1643), F(1118), L(3)

📄 [8/1560] 🔄 Processando: PP-FLJ_11_09_2020-AC..pdf
  ✅ Extraído: H(392), A(8527), F(1007), L(4)

📄 [9/1560] 🔄 Processando: pr_jhm_14_02_11.pdf
  ✅ Extraído: H(506), A(3785), F(666), L(2)

📄 [10/1560] 🔄 Processando: PT-UNS

In [12]:
df_v1 = pd.read_csv("../dados/json/_relatorio_processamento_v1.csv")
print("v1", sum(df_v1["fatores_count"]))  # 3892

df_v2 = pd.read_csv("../dados/json/_relatorio_processamento_v2.csv")
print("v2", sum(df_v2["fatores_count"]))  # 4281

df_novo = pd.read_csv("../dados/json/_relatorio_processamento.csv")
print("novo", sum(df_novo["fatores_count"]))  # 4006



v1 3892
v2 4281
novo 3979
